In [ ]:
import tensorflow as tf

import numpy as np
from tensorflow.keras.layers import LSTM, Bidirectional, GRU, TextVectorization, Dense, Embedding, Input, MultiHeadAttention, LayerNormalization, Layer
from tensorflow.keras.models import Model

In [ ]:
tf.random.set_seed(42)

In [ ]:
!wget https://www.manythings.org/anki/fra-eng.zip

--2023-07-30 02:27:32--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7420323 (7.1M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.08M  4.15MB/s    in 1.7s    

2023-07-30 02:27:35 (4.15 MB/s) - ‘fra-eng.zip’ saved [7420323/7420323]



In [ ]:
!unzip "/content/fra-eng.zip" -d "/content/dataset/"

Archive:  /content/fra-eng.zip
  inflating: /content/dataset/_about.txt  
  inflating: /content/dataset/fra.txt  


In [ ]:
downloaded_dataset = tf.data.TextLineDataset('/content/dataset/fra.txt')

In [ ]:
# Function to separate english text from french text
def selector(text_data):
  split_text = tf.strings.split(text_data, sep='\t')
  return {'input_1': split_text[0], 'input_2': 'starttoken ' + split_text[1]}, split_text[1] + ' endtoken'

In [ ]:
# Function to help create english and french vocabulary
def separator(text_data):
  split_text = tf.strings.split(text_data, sep='\t')
  return split_text[0], 'starttoken ' + split_text[1] + ' endtoken'

In [ ]:
selected_text = downloaded_dataset.map(selector)

In [ ]:
for i in selected_text.take(2):
  print(i)

({'input_1': <tf.Tensor: shape=(), dtype=string, numpy=b'Go.'>, 'input_2': <tf.Tensor: shape=(), dtype=string, numpy=b'starttoken Va !'>}, <tf.Tensor: shape=(), dtype=string, numpy=b'Va ! endtoken'>)
({'input_1': <tf.Tensor: shape=(), dtype=string, numpy=b'Go.'>, 'input_2': <tf.Tensor: shape=(), dtype=string, numpy=b'starttoken Marche.'>}, <tf.Tensor: shape=(), dtype=string, numpy=b'Marche. endtoken'>)


In [ ]:
separated_text = downloaded_dataset.map(separator)

In [ ]:
for i in separated_text.take(2):

  print(i)

(<tf.Tensor: shape=(), dtype=string, numpy=b'Go.'>, <tf.Tensor: shape=(), dtype=string, numpy=b'starttoken Va ! endtoken'>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Go.'>, <tf.Tensor: shape=(), dtype=string, numpy=b'starttoken Marche. endtoken'>)


In [ ]:
#How about we adapt a vectorizer on the separated text to get english and french vocabulary

#Firstly define global variables

VOCAB_SIZE = 20000
ENGLISH_SEQUENCE_LENGTH = 64
FRENCH_SEQUENCE_LENGTH = 64
EMBEDDING_DIM = 512
BATCH_SIZE = 64
HIDDEN_UNITS = 256

In [ ]:
english_vectorize_layer = TextVectorization(
    standardize = 'lower_and_strip_punctuation',
    max_tokens = VOCAB_SIZE,
    output_mode = 'int',
    output_sequence_length = ENGLISH_SEQUENCE_LENGTH
)

In [ ]:
french_vectorize_layer = TextVectorization(
    standardize = 'lower_and_strip_punctuation',
    max_tokens = VOCAB_SIZE,
    output_mode = 'int',
    output_sequence_length = FRENCH_SEQUENCE_LENGTH
)

In [ ]:
english = separated_text.map(lambda x,y:x)
english_vectorize_layer.adapt(english)

In [ ]:
french = separated_text.map(lambda x,y:y)
french_vectorize_layer.adapt(french)

In [ ]:
def vectorize(input, output):
  return {
     'input_1' : english_vectorize_layer(input['input_1']),
     'input_2' : french_vectorize_layer(input['input_2'])
  }, french_vectorize_layer(output)

In [ ]:
dataset = selected_text.map(vectorize)

In [ ]:
for i in dataset.take(5):
  print(i)

({'input_1': <tf.Tensor: shape=(64,), dtype=int64, numpy=
array([44,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])>, 'input_2': <tf.Tensor: shape=(64,), dtype=int64, numpy=
array([  2, 104,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])>}, <tf.Tensor: shape=(64,), dtype=int64, numpy=
array([104,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,

In [ ]:
dataset = dataset.shuffle(2048).unbatch().batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
dataset

<_PrefetchDataset element_spec=({'input_1': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'input_2': TensorSpec(shape=(None,), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
num_batches = 200000 / BATCH_SIZE

train_data = dataset.take(int(0.9*num_batches))
test_data = dataset.skip(int(0.9*num_batches))

In [ ]:
# Next, the most crucial part is to build an encoder-decoder architecture with bahdanau's attention mechanism

# The encoder block
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, lstm_units):
    super(Encoder, self).__init__()
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.lstm_units = lstm_units

  def build(self, input_shape):
    self.embedding = Embedding(self.vocab_size, self.embedding_dim)
    self.bidirectional_lstm = Bidirectional(LSTM(self.lstm_units, return_sequences=True))

  def call(self, x):
    x = self.embedding(x)
    output = self.bidirectional_lstm(x)

    return output



In [ ]:
#Create customized bahdanau attention class to be used as layer in the decoder block

class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()

    self.units = units

  def build(self, input_shape):
    self.dense1 = Dense(self.units)
    self.dense2 = Dense(self.units)
    self.dense = Dense(1)

  def call(self, previous_decoder_state, encoder_output):
    scores = self.dense(
                  tf.nn.tanh(
                        self.dense1(tf.expand_dims(previous_decoder_state, axis=-2)) +
                        self.dense2(encoder_output)

                            )
    )

    attention_weights = tf.nn.softmax(scores, axis=1)
    context_vector = attention_weights * encoder_output
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector

In [ ]:
# We now create the decoder block

class Decoder(tf.keras.Model):

  def __init__(self, vocab_size, embedding_dim, sequence_length, hidden_units):
    super(Decoder, self).__init__()

    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.sequence_length = sequence_length
    self.hidden_units = hidden_units

  def build(self, input_shape):
    self.embedding = Embedding(self.vocab_size, self.embedding_dim)
    self.attention = BahdanauAttention(self.hidden_units)
    self.gru = GRU(self.hidden_units, return_sequences=True, return_state=True)
    self.dense = Dense(self.vocab_size, activation='softmax')

  def call(self, x, previous_decoder_hidden_state, shifted_targets):
    outputs = []
    shifted_targets = self.embedding(shifted_targets)

    for time_step in range(self.sequence_length):
      context_vector = self.attention(previous_decoder_hidden_state, x)

      decoder_input = context_vector + shifted_targets[:, time_step]

      output, hidden_state = self.gru(tf.expand_dims(decoder_input, axis=1))

      outputs.append(output[:,0])

    final_output = tf.convert_to_tensor(outputs)
    final_output  = tf.transpose(final_output, perm=[1,0,2])

    final_output = self.dense(final_output)

    return final_output



In [ ]:
#Define parameters for model

#Encoder
input = Input(shape=(ENGLISH_SEQUENCE_LENGTH,), dtype='int64', name='input_1')
encoder = Encoder(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_UNITS)

encoder_output = encoder(input)

#Decoder
shifted_targets = Input(shape=(FRENCH_SEQUENCE_LENGTH,), dtype='int64', name='input_2')

decoder = Decoder(VOCAB_SIZE, EMBEDDING_DIM, FRENCH_SEQUENCE_LENGTH, HIDDEN_UNITS)

output = decoder(encoder_output, tf.zeros([1, HIDDEN_UNITS]), shifted_targets)


#Model
trans_model = Model([input, shifted_targets], output)
trans_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 encoder (Encoder)              (None, 64, 512)      11814912    ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 decoder (Decoder)              (None, 64, 20000)    16168737    ['encoder[0][0]',                
                                                                  'input_2[0][0]']            

In [ ]:
# Create function (method) that handles the positional encoding

def positional_encoding(d_model, sequence_length):

    """
    Computes the positional encoding that will be added to input embedding vectors

    Args

    -----

    d_model --> dtype - int
    sequence_length --> dtype - int

    """

    output = []

    for pos in range(sequence_length):

        #Initialize the positional encoding with zeros
        PE = np.zeros((d_model))

        for i in range(d_model):

            if i%2==0:
                PE[i] = np.sin(pos / (10000**((i) / d_model)))

            else:
                PE[i] = np.cos(pos / (10000**((i-1) / d_model)))

        output.append(tf.expand_dims(PE, axis=0))

    out = tf.concat(output, axis=0)
    out  = tf.expand_dims(out, axis=0)

    return tf.cast(out, dtype=tf.float32)

In [ ]:
#Embedding Layer

class Embeddings(Layer):

    def __init__(self, vocab_size, embedding_dim, sequence_length):

        super(Embeddings, self).__init__()

        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.sequence_length = sequence_length

        self.embedding_layer = Embedding(vocab_size, embedding_dim)

    def call(self, inputs):

        embedded_tokens = self.embedding_layer(inputs)
        position_encoding = positional_encoding(self.embedding_dim, self.sequence_length)

        final_embedding = position_encoding + embedded_tokens

        return final_embedding

    def mask(self, inputs, mask=None):

        mask = tf.math.not_equal(inputs, 0)



        return mask


In [ ]:
class TransformerEncoder(Layer):

    def __init__(self, embedd_dim, dense_dim, num_heads):

        super(TransformerEncoder, self).__init__()

        self.embedd_dim = embedd_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.supports_masking = True

    def build(self, input_shape):

        self.multi_head_attention = MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedd_dim)
        self.feed_forward = tf.keras.Sequential([Dense(self.dense_dim, activation='relu'), Dense(self.embedd_dim)])
        self.layer_norm_1 = LayerNormalization()
        self.layer_norm_2 = LayerNormalization()

    def call(self, inputs, mask=None):

        padding_mask = None

        if mask is not None:
            mask_1 = mask[:, :, tf.newaxis]
            mask_2 = mask[:, tf.newaxis, :]
            padding_mask = tf.cast(mask_1&mask_2, dtype=tf.int32)


        output_1 = self.multi_head_attention(query=inputs, key=inputs, value=inputs, attention_mask=padding_mask)

        projection = self.layer_norm_1(inputs + output_1)
        feed_forward_output = self.feed_forward(projection)

        final_output = self.layer_norm_2(projection + feed_forward_output)

        return final_output

In [ ]:
class TransformerDecoder(Layer):

    def __init__(self, embedd_dim, dense_dim, num_heads):

        super(TransformerDecoder, self).__init__()

        self.embedd_dim  = embedd_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.supports_masking = True

    def build(self, input_shape):

        self.multi_head_attention_1 = MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedd_dim)
        self.multi_head_attention_2 = MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedd_dim)
        self.feed_forward = tf.keras.Sequential([Dense(self.dense_dim, activation='relu'), Dense(self.embedd_dim)])
        self.layer_norm_1 = LayerNormalization()
        self.layer_norm_2 = LayerNormalization()
        self.layer_norm_3 = LayerNormalization()
        self.encoder = TransformerEncoder(self.embedd_dim, self.dense_dim, self.num_heads)

    def call(self, inputs, encoder_outputs, mask=None):

        padding_mask = None

        if mask is not None:
            mask_1 = mask[:, :, tf.newaxis]
            mask_2 = mask[:, tf.newaxis, :]
            padding_mask = tf.cast(mask_1&mask_2, dtype=tf.int32)


        attention_output_1 = self.multi_head_attention_1(query=inputs, key=inputs, value=inputs, attention_mask=padding_mask,
                                                       use_causal_mask=True)

        query_input = self.layer_norm_1(inputs + attention_output_1)

        attention_output_2 = self.multi_head_attention_2(query=query_input, key=encoder_outputs,
                                                         value=encoder_outputs, attention_mask=padding_mask)
        feed_forward_input = self.layer_norm_2(query_input + attention_output_2)

        feed_forward_output = self.feed_forward(feed_forward_input)

        final_output = self.layer_norm_3(feed_forward_input + feed_forward_output)

        return final_output

In [ ]:
EMBEDDING_DIM=128
D_FF=1024
NUM_HEADS=8
NUM_LAYERS=4
NUM_EPOCHS=20


In [ ]:
encoder_inputs=Input(shape=(None,), dtype="int64", name="input_1")
x = Embeddings(VOCAB_SIZE, EMBEDDING_DIM, ENGLISH_SEQUENCE_LENGTH)(encoder_inputs)

for _ in range(NUM_LAYERS):
 encoder_output=TransformerEncoder(EMBEDDING_DIM,D_FF,NUM_HEADS)(x)
encoder_outputs = encoder_output

print(encoder_outputs.shape)


decoder_inputs=Input(shape=(None,), dtype="int64", name="input_2")

x = Embeddings(VOCAB_SIZE, EMBEDDING_DIM, FRENCH_SEQUENCE_LENGTH)(decoder_inputs)
for i in range(NUM_LAYERS):
  x=TransformerDecoder(EMBEDDING_DIM,D_FF,NUM_HEADS)(x, encoder_outputs)


decoder_outputs=Dense(VOCAB_SIZE, activation="softmax")(x)

transformer = tf.keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)
transformer.summary()

(None, 64, 128)
Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embeddings (Embeddings)        (None, 64, 128)      2560000     ['input_1[0][0]']                
                                                                                                  
 embeddings_1 (Embeddings)      (None, 64, 128)      2560000     ['input_2[0][0]']                
                                                                        

In [ ]:
class BLEU(tf.keras.metrics.Metric):
    def __init__(self,name='bleu_score'):
        super(BLEU,self).__init__()
        self.bleu_score=0

    def update_state(self,y_true,y_pred,sample_weight=None):
      y_pred=tf.argmax(y_pred,-1)
      self.bleu_score=0
      for i,j in zip(y_pred,y_true):
        tf.autograph.experimental.set_loop_options()

        total_words=tf.math.count_nonzero(i)
        total_matches=0
        for word in i:
          if word==0:
            break
          for q in range(len(j)):
            if j[q]==0:
              break
            if word==j[q]:
              total_matches+=1
              j=tf.boolean_mask(j,[False if y==q else True for y in range(len(j))])
              break

        self.bleu_score+=total_matches/total_words

    def result(self):
        return self.bleu_score/BATCH_SIZE

In [ ]:
transformer.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(2e-4),) #metrics=[BLEU()],
    #run_eagerly=True)

In [ ]:
history=transformer.fit(
    train_data,
    validation_data=test_data.take(300),
    epochs=NUM_EPOCHS)

Epoch 1/20
2812/2812 [==============================] - 102s 27ms/step - loss: 0.5889 - val_loss: 0.2968
Epoch 2/20
2812/2812 [==============================] - 70s 25ms/step - loss: 0.2506 - val_loss: 0.2674
Epoch 3/20
2812/2812 [==============================] - 70s 25ms/step - loss: 0.2381 - val_loss: 0.2470
Epoch 4/20
2812/2812 [==============================] - 71s 25ms/step - loss: 0.2273 - val_loss: 0.2615
Epoch 5/20
2812/2812 [==============================] - 72s 25ms/step - loss: 0.2259 - val_loss: 0.2515
Epoch 6/20
2812/2812 [==============================] - 73s 26ms/step - loss: 0.2220 - val_loss: 0.2495
Epoch 7/20
2812/2812 [==============================] - 71s 25ms/step - loss: 0.2177 - val_loss: 0.2508
Epoch 8/20
2812/2812 [==============================] - 70s 25ms/step - loss: 0.2159 - val_loss: 0.2581
Epoch 9/20
2812/2812 [==============================] - 70s 25ms/step - loss: 0.2229 - val_loss: 0.2812
Epoch 10/20
2812/2812 [==============================] - 72s 25

In [ ]:
# transformer.evaluate(test_data)

In [ ]:
index_to_word={x:y for x, y in zip(range(len(french_vectorize_layer.get_vocabulary())),
                                   french_vectorize_layer.get_vocabulary())}

In [ ]:
def translator(english_sentence):
  tokenized_english_sentence=english_vectorize_layer([english_sentence])
  shifted_target='starttoken'

  for i in range(FRENCH_SEQUENCE_LENGTH):
    tokenized_shifted_target=french_vectorize_layer([shifted_target])
    output=transformer.predict([tokenized_english_sentence,tokenized_shifted_target], verbose=0)
    french_word_index=tf.argmax(output,axis=-1)[0][i].numpy()
    current_word=index_to_word[french_word_index]
    if current_word=='endtoken':
      break
    shifted_target+=' '+current_word
  return shifted_target[11:]

In [ ]:
translator('What makes you think that it is not true?')

'cest que tom'

In [ ]:
translator('Have you ever watched soccer under the rain?')

'[UNK]'

In [ ]:
translator("what's your name?")

'[UNK]'

In [ ]:
translator('She handed him the money')

'cest que tom'

In [ ]:
def positional_encoding(model_size,SEQUENCE_LENGTH):
  output=[]
  for pos in range(SEQUENCE_LENGTH):
    PE=np.zeros((model_size))
    for i in range(model_size):
      if i%2==0:
        PE[i]=np.sin(pos/(10000**(i/model_size)))
      else:
        PE[i]=np.cos(pos/(10000**((i-1)/model_size)))
    output.append(tf.expand_dims(PE,axis=0))
  out=tf.concat(output,axis=0)
  out=tf.expand_dims(out,axis=0)
  return tf.cast(out,dtype=tf.float32)

In [ ]:
class Embeddings(Layer):
  def __init__(self, sequence_length, vocab_size, embed_dim,):
    super(Embeddings, self).__init__()
    self.token_embeddings=Embedding(
        input_dim=vocab_size, output_dim=embed_dim)
    self.sequence_length = sequence_length
    self.vocab_size = vocab_size
    self.embed_dim = embed_dim

  def call(self, inputs):
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions=positional_encoding(
        self.embed_dim,self.sequence_length)
    return embedded_tokens + embedded_positions

  def compute_mask(self, inputs, mask=None):
    return tf.math.not_equal(inputs, 0)

In [ ]:
class TransformerEncoder(Layer):
    def __init__(self, embed_dim, dense_dim, num_heads,):
        super(TransformerEncoder, self).__init__()
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim,
        )
        self.dense_proj=tf.keras.Sequential(
            [Dense(dense_dim, activation="relu"),Dense(embed_dim),]
        )
        self.layernorm_1 = LayerNormalization()
        self.layernorm_2 = LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):

      padding_mask = None
      if mask is not None:
        mask1 = mask[:, :, tf.newaxis]
        mask2 = mask[:,tf.newaxis, :]
        padding_mask = tf.cast(mask1&mask2, dtype="int32")

      attention_output = self.attention(
          query=inputs, key=inputs,value=inputs,attention_mask=padding_mask
      )

      proj_input = self.layernorm_1(inputs + attention_output)
      proj_output = self.dense_proj(proj_input)
      return self.layernorm_2(proj_input + proj_output)

In [ ]:
class TransformerDecoder(Layer):
  def __init__(self, embed_dim, latent_dim, num_heads,):
    super(TransformerDecoder, self).__init__()
    self.embed_dim = embed_dim
    self.latent_dim = latent_dim
    self.num_heads = num_heads
    self.attention_1=MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim
    )
    self.attention_2=MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim
    )
    self.dense_proj = tf.keras.Sequential(
        [Dense(latent_dim, activation="relu"),Dense(embed_dim),]
    )
    self.layernorm_1=LayerNormalization()
    self.layernorm_2=LayerNormalization()
    self.layernorm_3=LayerNormalization()
    self.supports_masking = True

  @tf.function
  def call(self, inputs, encoder_outputs, mask=None):

    if mask is not None:
      padding_mask = None
      mask1 = mask[:, :, tf.newaxis]
      mask2 = mask[:,tf.newaxis, :]
      padding_mask = tf.cast(mask1&mask2, dtype="int32")
      causal_mask=tf.linalg.band_part(tf.ones([tf.shape(inputs)[0],tf.shape(inputs)[1],
                                               tf.shape(inputs)[1]],dtype=tf.int32),-1,0)
      combined_mask=tf.minimum(padding_mask,causal_mask)

    attention_output_1 = self.attention_1(
        query=inputs,key=inputs,value=inputs,
        attention_mask=causal_mask,

    )
    out_1 = self.layernorm_1(inputs + attention_output_1)

    attention_output_2,scores= self.attention_2(
        query=out_1,key=encoder_outputs,value=encoder_outputs,
        attention_mask=combined_mask,
    )
    out_2 = self.layernorm_2(out_1 + attention_output_2)

    proj_output = self.dense_proj(out_2)
    return self.layernorm_3(out_2 + proj_output)

In [ ]:
encoder_inputs=Input(shape=(None,), dtype="int64", name="input_1")
x = Embeddings(ENGLISH_SEQUENCE_LENGTH,VOCAB_SIZE,EMBEDDING_DIM)(encoder_inputs)

for _ in range(NUM_LAYERS):
  encoder=TransformerEncoder(EMBEDDING_DIM,D_FF,NUM_HEADS)(x)
encoder_outputs=encoder

decoder_inputs=Input(shape=(None,), dtype="int64", name="input_2")

x = Embeddings(FRENCH_SEQUENCE_LENGTH,VOCAB_SIZE,EMBEDDING_DIM)(decoder_inputs)
for i in range(NUM_LAYERS):
  x=TransformerDecoder(EMBEDDING_DIM,D_FF,NUM_HEADS)(x, encoder_outputs)
decoder_outputs=Dense(VOCAB_SIZE, activation="softmax")(x)

transformer = tf.keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)
transformer.summary()

OperatorNotAllowedInGraphError: ignored